# Damage Proxy Map

In [1]:
import fiona
import json
import rasterio
import numpy as np
from glob import glob
from os.path import join, isfile, isdir
from os import mkdir
from matplotlib import pyplot as plt
from rasterio import features
from rasterio.mask import mask
from rasterio.merge import merge
from rasterstats import zonal_stats
from shapely.geometry import shape, box, mapping
from shapely import unary_union
import numpy as np

## Define paths to the required files

In [2]:
WORKDIR = "/Users/jorgemartinez/data/Automation/"
OUTPUT_DIR = "/Users/jorgemartinez/data/Automation/outputs"

RASTER_DPM_FOLDER = join(WORKDIR, "DPM", "*.tif")

BUILDING_FOOTPRINTS_PATH = join(WORKDIR, "Building Footprints_byMicrosoft", "syria_buildingfootprints.shp")
BOUNDARIES_PATH = join(WORKDIR, "Admin_bound_level2", "syria_admin2.shp")
SHAKEMAP_BOUNDARIES_PATH = join(WORKDIR, "MISC", "shakemap.shp")
DLR_FOOTPRINTS_PATH = join(WORKDIR, "DLR", "syria84.shp")

BOUNDARIES_NAME_FIELD = "adm2_name"

RASTER_MOSAIC = join(OUTPUT_DIR, "mosaic.tif")
WORLDPOP_RASTER_PATH = join(WORKDIR, "WorldPop2020", "syr_ppp_2020_UNadj.tif")
VECTOR_DPM = join(OUTPUT_DIR, "vector_dpm")
LABELED_BUILDINGS = join(OUTPUT_DIR, "labeled_buildings")
LABELED_DLR = join(OUTPUT_DIR, "labeled_dlr")

if isdir(OUTPUT_DIR) is False:
    mkdir(OUTPUT_DIR)

## Merge rasters

In [4]:
if isfile(RASTER_MOSAIC) is False:
    rasters = [rasterio.open(f) for f in glob(RASTER_DPM_FOLDER)]
    print(f"Found {len(rasters)} raster files")
    mosaic, out_transform = merge(rasters, method="max")
    _, height, width = mosaic.shape

    metadata = rasters[0].meta.copy()
    out_meta = {**metadata, "height": height, "width": width, "transform": out_transform}

    with rasterio.open(RASTER_MOSAIC, "w", **out_meta) as dest:
        dest.write(mosaic)

# DLR urban polygons based analysis (Sirio's workflow)

## Apply zonal statistics using DLR polygons and get count of 255 pixels.

In [5]:
def count_threshold(mask):
    return np.count_nonzero(mask == 255)

stat_name = "count_px"
stats = zonal_stats(DLR_FOOTPRINTS_PATH, RASTER_MOSAIC, add_stats={stat_name: count_threshold}, geojson_out=True)

# Remove the geometries without threshold_count.
filtered_dlr_polygons = [d for d in stats if d["properties"][stat_name] > 0]

/Users/jorgemartinez/venvs/dpm/lib/python3.11/site-packages/rasterstats/io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [17]:
filtered_dlr_polygons[0]

{'type': 'Feature',
 'id': '4',
 'properties': OrderedDict([('FID', 4),
              ('min', 0.0),
              ('max', 255.0),
              ('mean', 7.183098591549296),
              ('count', 71),
              ('count_px', 2),
              ('sum', 10.125910758972168)]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(38.197623522159795, 36.89515619171493),
    (38.196765560882575, 36.89537620877864),
    (38.19642895582934, 36.895864141357784),
    (38.196061558914316, 36.89578176654801),
    (38.19620183295136, 36.895156191714925),
    (38.19611103643582, 36.894751949837065),
    (38.19628762907909, 36.89396399398921),
    (38.195574152691805, 36.89369871025689),
    (38.194818567775584, 36.89585466693875),
    (38.19430905457536, 36.89620627315543),
    (38.196144196902374, 36.896677362323004),
    (38.196875569414445, 36.89620100958932),
    (38.197194541521185, 36.896663150694486),
    (38.19763089115237, 36.89688316775821),
    (38.19800302527689, 36.89634417858774),
 

## Apply zonal statistics to the DLR filtered polygons with the World Pop raster

In [6]:
stats = zonal_stats(filtered_dlr_polygons, WORLDPOP_RASTER_PATH, stats=["sum"], geojson_out=True)

## Label the DLR polygon using the admin boundary.

In [7]:
shapely_features = [{
    "properties": feat["properties"],
    "shape": shape(feat["geometry"])
} for feat in stats]

def assign_name(feature, shapely_features):
    name = feature["properties"][BOUNDARIES_NAME_FIELD]    
    boundary_shape = shape(feature["geometry"])

    filtered_dlr_polygons = [f for f in shapely_features if boundary_shape.contains(f["shape"])]
    print(f"Found {len(filtered_dlr_polygons)} dlr polygons in {name}")
    
    return {
        "name": name,
        "dlr_polygons": filtered_dlr_polygons,
    }


with fiona.open(BOUNDARIES_PATH, "r") as shapefile:
    dlr_polygons_with_names = [assign_name(feature, shapely_features) for feature in shapefile]

Found 1 dlr polygons in Dreikish
Found 0 dlr polygons in Duma
Found 299 dlr polygons in Hama
Found 96 dlr polygons in Harim
Found 331 dlr polygons in Homs
Found 181 dlr polygons in Idleb
Found 0 dlr polygons in Izra'
Found 59 dlr polygons in Masyaf
Found 231 dlr polygons in Menbij
Found 45 dlr polygons in Muhradah
Found 0 dlr polygons in Qatana
Found 0 dlr polygons in Ras Al Ain
Found 0 dlr polygons in Rural Damascus
Found 0 dlr polygons in Al Qutayfah
Found 0 dlr polygons in Shahba
Found 290 dlr polygons in A'Zaz
Found 26 dlr polygons in Al-Qardaha
Found 32 dlr polygons in Safita
Found 0 dlr polygons in Abu Kamal
Found 212 dlr polygons in Afrin
Found 54 dlr polygons in Ain Al Arab
Found 60 dlr polygons in Al-Qusayr
Found 0 dlr polygons in Tell Abiad
Found 61 dlr polygons in Al-Haffa
Found 0 dlr polygons in Al-Hasakeh
Found 0 dlr polygons in Al-Malikeyyeh
Found 505 dlr polygons in Al Bab
Found 0 dlr polygons in Al Fiq
Found 270 dlr polygons in Al Ma'Ra
Found 52 dlr polygons in Al Makhr

In [8]:
schema = {
    "geometry": "Polygon",
    "properties": {
        stat_name: "int",
        "name": "str",
        "population_sum": "float"
    }
}
    
with fiona.collection(LABELED_DLR, "w", schema=schema, driver="ESRI Shapefile") as output:
    for item in dlr_polygons_with_names:    
        adm_name = item["name"]
        for feature in item["dlr_polygons"]:        
            properties = {
                stat_name: feature["properties"][stat_name],
                "population_sum": feature["properties"]["sum"],
                "name": adm_name
            }

            output.write({
                "type": "Feature",
                "geometry": mapping(feature["shape"]),
                "properties": properties
            })

    

In [15]:
dlr_polygons_with_names[6]

{'name': "Izra'", 'dlr_polygons': []}

## Clipping raster using the country boundaries as bounding box

In [ ]:
with rasterio.open(RASTER_MOSAIC, "r") as dataset:
    with fiona.open(BOUNDARIES_PATH, "r") as shapefile:
        bbox = box(*shapefile.bounds)
    out_image, out_transform = mask(dataset, [bbox], crop=True)
    _, height, width = out_image.shape
    band_4 = out_image[3, :, :]
    
    # Transform to vector data.
    condition = (band_4 == 255).astype(np.uint8)
    dpm_polygons = [shape(coords) for coords, value in
                    features.shapes(condition, transform=out_transform, connectivity=8) if value != 0]

In [ ]:
len(dpm_polygons)

## Saving dpm polygons

In [ ]:
schema = {
    "geometry": "Polygon",
    "properties": {}
}

def write_shape(name, polygon, output):
    output.write({
        "type": "Feature",
        "geometry": polygon,
        "properties": {"name": name}
    })
    
with fiona.collection(VECTOR_DPM, "w", schema=schema, driver="ESRI Shapefile") as output:
    for polygon in dpm_polygons:    
        output.write({
            "type": "Feature",
            "geometry": mapping(polygon),
            "properties": {}
        })


## Filter buildings within the dpm polygons set

In [ ]:
dpm_buildings = []
buildings_shp = fiona.open(BUILDING_FOOTPRINTS_PATH, "r")
for polygon in dpm_polygons:    
    filtered_buildings = buildings_shp.filter(bbox=(polygon.bounds))
    for building in filtered_buildings:
        fid = building.properties["FID_1"]        
        building_shape = shape(building.geometry)
        if building_shape.intersects(polygon) is False:
            continue
        """
        if fid in [f["id"] for f in dpm_buildings]:
            continue
        """
        matched_building = {"id": fid, "shape": building_shape}
        dpm_buildings.append(matched_building)
buildings_shp.close()
dpm_buildings = [d["shape"] for d in dpm_buildings]

In [ ]:
len(dpm_buildings)

## Label buildings using admin level 2

In [ ]:
def assign_name(feature, dpm_buildings):
    name = feature["properties"][BOUNDARIES_NAME_FIELD]    
    boundary_shape = shape(feature["geometry"])

    filtered_buildings = [f for f in dpm_buildings if boundary_shape.contains(f)]
    print(f"Found {len(filtered_buildings)} buildings in {name}")

    return {
        "name": name,
        "buildings": filtered_buildings,
    }
    
with fiona.open(BOUNDARIES_PATH, "r") as boundaries_shp:
    dpm_buildings_with_label = [assign_name(boundary, dpm_buildings) for boundary in boundaries_shp]

## Saving result to shapefile

In [ ]:
schema = {
    "geometry": "Polygon",
    "properties": {"name": "str"}
}
with fiona.collection(LABELED_BUILDINGS, "w", schema=schema, driver="ESRI Shapefile") as output:
    for item in dpm_buildings_with_label:
        name = item["name"]
        for item_geom in item["buildings"]:
            output.write({
                "type": "Feature",
                "geometry": mapping(item_geom),
                "properties": {"name": name}
            })